# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests, json, re
import gmaps
import os

# Import API keys
from config import (ow_key, gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../WeatherPy/output_data/cities.csv"
weather_data_df = pd.read_csv(file, encoding="utf-8")

weather_data_df.drop('Degrees From Equator', inplace=True, axis=1)
weather_data_df.head()


,City,Country Code,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date
0,Klaksvik,FO,62.2266,-6.5890,3.0,87.0,20.0,7.72,1.619491e+09
1,Constitucion,CL,-35.3333,-72.4167,14.0,62.0,0.0,2.86,1.619491e+09
2,Rawson,AR,-43.3002,-65.1023,17.0,44.0,62.0,3.09,1.619491e+09
3,Cidreira,BR,-30.1811,-50.2056,16.0,79.0,1.0,4.25,1.619491e+09
4,Punta Arenas,CL,-53.1500,-70.9167,6.0,100.0,90.0,1.54,1.619491e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data_df[["Latitude", "Longitude"]]

humidity = weather_data_df["Humidity"]

In [4]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(0.0, 30.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=weather_data_df["Humidity"].max(),
                                 point_radius = 3)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
ideal_vacay_spots_df = weather_data_df.loc[(weather_data_df['Temperature'] >= 22) &  
                                        (weather_data_df['Temperature'] <= 26) &
                                        (weather_data_df['Wind Speed'] <= 5) & 
                                        (weather_data_df['Cloudiness'] == 0)
                                        ].reset_index(drop=True)

ideal_vacay_spots_df[['Hotel Name','Country']] = ''
ideal_vacay_spots_df

,City,Country Code,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Country
0,Maceio,BR,-9.6658,-35.7353,23.0,94.0,0.0,1.03,1.619491e+09,,
1,Carnarvon,AU,-24.8667,113.6333,26.0,47.0,0.0,1.54,1.619491e+09,,
2,Inhambane,MZ,-23.8650,35.3833,22.0,90.0,0.0,2.41,1.619491e+09,,
3,Guaratinga,BR,-16.5667,-39.5667,24.0,78.0,0.0,2.57,1.619491e+09,,
4,Dubbo,AU,-32.2500,148.6167,22.0,40.0,0.0,2.06,1.619491e+09,,
5,Perth,AU,-31.9333,115.8333,26.0,43.0,0.0,2.06,1.619491e+09,,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.


#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in ideal_vacay_spots_df.iterrows():

    params = {
    "location": f"{row[2]},{row[3]}",
    "radius": 5000,
    "type": "lodging",
    "keyword" : "hotel",
    "language" : "en",
    "key": gkey
    }
    
    try:
        first_hotel = requests.get(base_url, params=params).json()['results'][0]
        ideal_vacay_spots_df.at[index, 'Hotel Name'] = first_hotel['name']
        country, = re.findall('\s.*,\s(.*?)$', first_hotel['plus_code']['compound_code'])
        ideal_vacay_spots_df.at[index, 'Country'] = country
    except (KeyError, ValueError):
                ideal_vacay_spots_df.at[index, 'Country'] = ''

In [13]:
ideal_vacay_spots_df

,City,Country Code,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Country
0,Maceio,BR,-9.6658,-35.7353,23.0,94.0,0.0,1.03,1.619491e+09,Meridiano Hotel,Brazil
1,Carnarvon,AU,-24.8667,113.6333,26.0,47.0,0.0,1.54,1.619491e+09,Hospitality Carnarvon,Australia
2,Inhambane,MZ,-23.8650,35.3833,22.0,90.0,0.0,2.41,1.619491e+09,Hotel Casa Do Capitão,Mozambique
3,Guaratinga,BR,-16.5667,-39.5667,24.0,78.0,0.0,2.57,1.619491e+09,Hotel Caraíva,Brazil
4,Dubbo,AU,-32.2500,148.6167,22.0,40.0,0.0,2.06,1.619491e+09,Quest Dubbo,Australia
5,Perth,AU,-31.9333,115.8333,26.0,43.0,0.0,2.06,1.619491e+09,COMO The Treasury,Australia


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in ideal_vacay_spots_df.iterrows()]
locations = ideal_vacay_spots_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
# Create a map using state centroid coordinates to set markers

markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))